In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np

In [2]:
def DataCreation(data_size,start,end):
    input_data = []
    output_data = []
    input_size = torch.randint(start,end,(data_size,))
    for i in range(data_size):
        first_layer = torch.rand(input_size[i])
        x_0 = torch.randint(0,input_size[i],(1,))
        x_1 = torch.randint(1,input_size[i],(1,))
        if (x_1 == x_0):
            x_1 = x_1 - 1
        second_layer = torch.zeros(input_size[i])
        second_layer[x_0] = 1
        second_layer[x_1] = 1       
        input_data.append(torch.stack((first_layer,second_layer)))
        output_data.append(first_layer[x_0] + first_layer[x_1])
    return(input_data,output_data,data_size)

In [3]:
input_data,output_data,data_size = DataCreation(5000,2,10)
test_input,test_output,test_data_size = DataCreation(1000,2,14)

In [4]:
def train_loop(model, loss_fn, optimizer,data_size,input_data,output_data):
    loss_m = 0
    for i in range(data_size):
# Compute prediction
        X = input_data[i]
        y = output_data[i]
        pred = model(X)
# Calculating loss
        loss = loss_fn(pred, y)
# Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_m = loss_m + loss.detach()
    return loss_m/data_size

In [31]:
class LSTM(nn.Module):
    h = 5
    C = 5
    def __init__(self): 
        super().__init__()
        self.h0 = (torch.zeros(self.h,1))
        self.C0 = (torch.zeros(self.C,1))
        
#         variables
        self.Wf = nn.Parameter(torch.randn(self.h,self.h+2)/np.sqrt(self.h + 2))
        self.bf = nn.Parameter(torch.randn(self.h,1))
        self.Wi = nn.Parameter(torch.randn(self.h,self.h + 2)/np.sqrt(self.h + 2))
        self.bi = nn.Parameter(torch.randn(self.h,1))
        self.Wc = nn.Parameter(torch.randn(self.h,self.h + 2)/np.sqrt(self.h + 2))
        self.bc = nn.Parameter(torch.randn(self.h,1))
        self.Wo = nn.Parameter(torch.randn(self.h,self.h + 2)/np.sqrt(self.h + 2))
        self.bo = nn.Parameter(torch.randn(self.h,1))
        self.Wy = nn.Parameter(torch.randn(1,self.h))
        self.by = nn.Parameter(torch.randn(1,1))
    def forward(self,xt):
        self.ht = self.h0
        self.Ct = self.C0        
        Tanh = nn.Tanh() # Tanh non-linearity
        Sigmoid = nn.Sigmoid() # Sigmoid non-linearity
        for i in range(len(xt)):
            cathx = torch.cat((self.ht,xt[:,i].unsqueeze(1)))
            self.ft = Sigmoid(self.Wf@(cathx) + self.bf)
            self.it = Sigmoid(self.Wi@(cathx) + self.bi)
            self.C_t = Tanh(self.Wc@(cathx) + self.bc)
            self.Ct = self.ft*self.Ct + self.it*self.C_t
            self.ot = Sigmoid(self.Wo@(cathx) + self.bo)
            self.ht = self.ot*Tanh(self.Ct)
        y = self.Wy@self.ht + self.by
        return y[0]
    

In [32]:
model_2 = LSTM()
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model_2.parameters(), lr=0.02)
LSTM_loss = torch.empty([1])
current_loss = 100
epochs = 0
while True:
    if (
#         abs(current_loss) > 0.05 and 
        epochs < 10) :
            current_loss = train_loop(model_2, loss_fn, optimizer,data_size,input_data,output_data)
            print("Epoch {}:{}".format(epochs+1,current_loss))
            LSTM_loss = torch.cat((LSTM_loss,torch.tensor([current_loss])))
            epochs = epochs + 1
    else:
        break
print("Done!")

Epoch 1:0.13900193572044373
Epoch 2:0.11710097640752792
Epoch 3:0.10268054157495499
Epoch 4:0.09985484927892685
Epoch 5:0.09862888604402542
Epoch 6:0.09793183952569962
Epoch 7:0.0975293442606926
Epoch 8:0.09728448837995529
Epoch 9:0.09712030738592148
Epoch 10:0.09699955582618713
Done!


In [ ]:
        ht = self.h0
        Ct = self.C0
        for i in range(len(xt)):
            cathx = torch.cat((ht,xt[:,i].unsqueeze(1)))
            ft = torch.sigmoid(self.Wf@(cathx) + self.bf)
            it = torch.sigmoid(self.Wi@(cathx) + self.bi)
            C_t = torch.tanh(self.Wc@(cathx) + self.bc)
            Ct = (ft*Ct) + (it*C_t)
            ot = torch.sigmoid(self.Wo@(cathx) + self.bo)
            ht = (ot*torch.tanh(Ct))
        y = self.Wy@ht + self.by

In [7]:
        self.ht = self.h0
        self.Ct = self.C0        
        Tanh = nn.Tanh() # Tanh non-linearity
        Sigmoid = nn.Sigmoid() # Sigmoid non-linearity
        for i in range(len(xt)):
            cathx = torch.cat((self.ht,xt[:,i].unsqueeze(1)))
            self.ft = Sigmoid(self.Wf@(cathx) + self.bf)
            self.it = Sigmoid(self.Wi@(cathx) + self.bi)
            self.C_t = Tanh(self.Wc@(cathx) + self.bc)
            self.Ct = self.ft*self.Ct + self.it*self.C_t
            self.ot = Sigmoid(self.Wo@(cathx) + self.bo)
            self.ht = self.ot*Tanh(self.Ct)
        y = self.Wy@self.ht + self.by

NameError: name 'self' is not defined

In [ ]:
           concat_vec = torch.cat((self.h_t,input_))
           self.f_t = Sigmoid(self.Wf@concat_vec + self.bias_f) # To do selective erase of prev hidden state
           self.i_t = Sigmoid(self.Wi@concat_vec + self.bias_i) # To do selective read from prev hidden state
            
           cand = Tanh(self.Wc@concat_vec + self.bias_c) # Candidate cell state
           
           self.C_t = self.C_t*self.f_t + self.i_t*cand # New cell state
           self.o_t = Sigmoid(self.Wo@concat_vec + self.bias_o) # To do selective write on prev hidden write
           self.h_t = Tanh(self.C_t)*self.o_t # New hidden state
        
        self.y_smp = self.Wy@self.h_t + self.bias_y # Output predicted by model

In [33]:
 self.h_t = self.h_0
        self.C_t = self.C_0
        
        Tanh = nn.Tanh() # Tanh non-linearity
        Sigmoid = nn.Sigmoid() # Sigmoid non-linearity

        for input_ in input_sq:
            
           input_ = input_.view(-1,1) # Adjusting the input to appropriate shape
        
           concat_vec = torch.cat((self.h_t,input_)) # Concatenating the hidden state and input  
           
           self.f_t = Sigmoid(self.Wf@concat_vec + self.bias_f) # To do selective erase of prev hidden state
           self.i_t = Sigmoid(self.Wi@concat_vec + self.bias_i) # To do selective read from prev hidden state
            
           cand = Tanh(self.Wc@concat_vec + self.bias_c) # Candidate cell state
           
           self.C_t = self.C_t*self.f_t + self.i_t*cand # New cell state
           self.o_t = Sigmoid(self.Wo@concat_vec + self.bias_o) # To do selective write on prev hidden write
           self.h_t = Tanh(self.C_t)*self.o_t # New hidden state
        
        self.y_smp = self.Wy@self.h_t + self.bias_y # Output predicted by model
        

IndentationError: unexpected indent (1384167505.py, line 2)